In [1]:
pip install requests
pip install selenium
pip install beautifulsoup4 openpyxl
pip install webdriver-manager
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [70]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook
import re
import time
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [37]:
import openpyxl

In [66]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [73]:
# Chrome 옵션 설정
options = Options()

In [74]:
# WebDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [75]:
# 파일 경로 설정
file_path = '제품 가격zz.xlsx'

# 엑셀 파일 열기
wb = openpyxl.load_workbook(file_path)
ws = wb.active

In [76]:
# 웹사이트 로그인 페이지 열기
driver.get("https://www.dalrat.com/admin")
time.sleep(5)  # 실제 페이지 로드 시간에 따라 조정

In [77]:
# 로그인 필드에 자격 증명 입력 (가정)
username = driver.find_element(By.ID, "username")
password = driver.find_element(By.ID, "login")
username.send_keys("dalrat")
password.send_keys("dalrat456!!")
password.send_keys(Keys.RETURN)
time.sleep(10)  # 실제 페이지 로드 시간에 따라 조정

In [78]:
driver.get("https://www.dalrat.com/admin/catalog/product/index/key/7320a92918c5026ca8aa126a87abfeff1a70ff79812c085eebb4c74174de9d85/")
time.sleep(5)

In [79]:
# 품목명 읽기
product_names = [cell.value for cell in ws['B'][1:]] 
# 한글만 추출
korean_product_names = [re.sub('[^가-힣]', '', name) for name in product_names if name is not None]

In [80]:
# Chrome 옵션 설정
options = Options()

In [81]:
for row, name in enumerate(korean_product_names, start=2):  # 2번째 행부터 시작
    search_input = driver.find_element(By.ID, "fulltext")  # 검색 필드 ID
    search_input.clear()
    search_input.send_keys(name)
    search_input.send_keys(Keys.ENTER)

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "img.admin__control-thumbnail")))  # 요소가 나타날 때까지 최대 10초 기다림
        img_element = driver.find_element(By.CSS_SELECTOR, "img.admin__control-thumbnail")
        img_src = img_element.get_attribute('src')
        ws['A' + str(row)].value = img_src  # 엑셀의 'A열'에 이미지 링크 저장
    except (NoSuchElementException, TimeoutException):
        print(f"이미지를 찾을 수 없습니다: {name}")
        ws['A' + str(row)].value = "이미지 없음"  # 이미지가 없을 경우에는 '이미지 없음'으로 저장

    time.sleep(3)  # 다음 검색 전에 잠시 대기

KeyboardInterrupt: 

In [72]:
wb.save('제품 가격zz.xlsx')
driver.quit()